<h5 style='text-align: right;'> Universidad de los Andes  &emsp; <br>  Profesor: Juan Fernando Pérez  &emsp;<br> Instructora: Natalia Pacheco  &emsp; <br> Primer semestre de 2023 &emsp; </h5>
<h4 style="text-align:center;">TALLER EN CLASE  <br> Redes Bayesianas </h4> 


# Problema de Monty Hall

En un programa concurso Ud puede ganar un carro. Para hacerlo debe seleccionar una de tres puertas. Detrás de una de ellas está el carro y si la selecciona se gana el carro. Tras su selección de una de las puertas, el animador del programa abrirá otra de las puertas, detrás de la que hay una cabra. Recuerde que el animador sabe qué hay detrás de cada puerta. Tras abrir la otra puerta, el animador le da la posibilidad de cambiar de puerta, ¿Ud se cambia o no de puerta?

 Para decidir si debe cambiar o no de puerta UD ha decidido modelar este problema como una Red Bayesiana.
  
  <p align="center"><image src="https://github.com/Pacheco-Carvajal/Taller_Redes/blob/main/red1.png"></p>

Donde $U, C $ y $A$ son tres variables aleatorias que representan la puerta seleccionada por Ud, la puerta detrás de la cual está el carro, y la puerta que selecciona el animador, respectivamente. Las tres variables tienen espacio de estados $\{1,2,3\}$.

1. Responda las siguientes preguntas:

a). ¿Tiene esta red estructuras en V? Si las tiene, ¿Qué implicaciones tienen en la influencia entre las variables de la red?

b). ¿Qué tipo de análisis considera es más relevante en este caso? ¿Causal, evidencial, intercausal? ¿Por qué?

c). ¿Qué independencias condicionales captura esta red?


2. El carro ha sido ubicado al azar detrás de alguna de las puertas, y Ud selecciona una puerta al azar inicialmente. Modele este comportamiento con las distribuciones de probabilidad P(U) y P(C) 

\\begin{array}{cccc}
     & \text{1} & \text{2} & \text{3}  \\
   P(U)=  &  \left[1/3 \right. & 1/3 & \left.1/3 \right]
\end{array}


\\begin{array}{cccc}
     & \text{1} & \text{2} & \text{3}  \\
   P(C)=  &  \left[1/3 \right. & 1/3 & \left.1/3 \right]
\end{array}

3. El comportamiento del Animador (la puerta que decide mostrar) depende tanto de su selección como de la puerta donde se encuentra el carro. Modele el comportamiento del Animador con la distribución condicional de probabilidad $P(A|C,U)$ (note que esta representación es estocástica por columnas)

In [ ]:
from tkinter import Image
Image(filename='Tabla.jpg',width=600)

4. En su editor de python cree un nuevo archivo monty.py e incluya los objetos BayesianNetwork y Tabular CPD

In [ ]:
#objetos BayesianNetwork y TabularCPD
from pgmpy.models import BayesianNetwork
from pgmpy.factors.discrete import TabularCPD

5. Defina ahora la estructura de la red incluyendo los arcos y nodos

In [ ]:
modelo = BayesianNetwork ([("C", "A") , ("U", "A")])

6. Defina las CPDs de $C$ y $U$

In [ ]:
#Definir las CPD's de C, U y A
cpd_c = TabularCPD(variable="C", variable_card=3, values=[[0.33], [0.33], [0.33]])
cpd_u = TabularCPD(variable="U", variable_card=3, values=[[0.33], [0.33], [0.33]])

7. Defina la CPD de A

In [ ]:
cpd_a = TabularCPD(
    variable="A",
    variable_card=3,
    values=[
        [0, 0, 0, 0, 0.5, 1, 0, 1, 0.5],
        [0.5, 0, 1, 0, 0, 0, 1, 0, 0.5],
        [0.5, 1, 0, 1, 0.5, 0, 0, 0, 0],
    ],
    evidence=["C", "U"],
    evidence_card=[3, 3],
)

8. Asocie las 3 CPDs a su modelo

In [ ]:
modelo.add_cpds(cpd_c, cpd_u, cpd_a)

9. Revise que su modelo este completo

In [ ]:
modelo.check_model()

10. Importe la clase VariableElimination del paquete de Inferencia y cree un objeto de esta clase.

In [ ]:
from pgmpy.inference import VariableElimination

infer = VariableElimination(modelo)

11. Suponga que usted selecciona la puerta 1 y el animador la puerta 3. ¿Cuál es la probabilidad de que el carro este detrás de cada una de las puertas?

**Nota:** Note que los valores se indexan desde 0, luego la puerta 1 corresponde al elemento 0, la 2 al elemento 1, y la 3 al elemento 2.

In [ ]:
posterior_p = infer.query(["C"], evidence={"U": 0, "A": 1})
print(posterior_p)

12. Interprete los resultados. ¿Es mejor cambiarse de puerta o no? 
- En este caso, lo mejor es cambiarse de puertas pues aumenta la probabilidad de ganarse el carro, pasa de 1/3 a 2/3.

13. Modifique la evidencia para considerar otros casos de selección de puertas, tanto la que Ud selecciona como la que el animador abre ¿Cómo interpreta estos resultados?

In [ ]:
posterior_p2 = infer.query(["C"], evidence={"U": 1, "A": 0})
print(posterior_p2)

- En este caso, al igual que en el caso anterior lo mejor sería cambiarse de puerta pues aumenta la probabilidad de 1/3 a 2/3 de ganarse el carro si se cambia la selección.

14. Compare ahora con el caso en que solo tiene como evidencia la puerta que Ud seleccionó.

In [ ]:
posterior_p = infer.query(["C"], evidence={"U": 0})
print(posterior_p)

* En este caso al conocer únicamente la elección del participante la probabilidad de que el carro se encuentre detrás de cada una de las puertas es la misma dado que no tenemos información adicional.

# Caso de aplicación: Confiabilidad

En la planeación de actividades de mantenimiento y presupuesto, así como para la definición de estrategias de mitigación del riesgo, es importante la estimación de la confiabilidad o la probabilidad de falla de un sistema. Por ello, se desea estimar el **nivel de confiabilidad** de un equipo durante el próximo mes con base en un conjunto de variables relevantes.

Debido a la estructura del problema y al propósito del análisis, se decidió modelar las relaciones entre las variables por medio de una Red Bayesiana.

Con tal fin, se identificaron las variables que influyen de manera significativa en la confiabilidad del equipo, se determinaron estados posibles para cada una de ellas y, con base en la experiencia previa y en juicios emitidos por expertos en el manejo de este equipo, se definieron las relaciones entre las variables y las probabilidades condicionales que las modelan apropiadamente.

Variables

-	**Historial Anormalidad** : el equipo puede tener o no tener registro de posibles anormalidades de funcionamiento en su hoja de vida. 
-	**Condiciones de Uso**: el equipo puede encontrarse en condiciones de uso normales o en sobreuso (condiciones extremas), de acuerdo con su diseño. 
-	**Análisis de Fallas**: hace referencia a la valoración de los daños presentados por el equipo previamente. El resultado de dicha valoración puede ser leves o graves.
-	**No. de Fallas Latentes**: representa el número de fallas posibles en el equipo, el cual puede ser bajo, medio o alto. Las fallas latentes dependen del Historial de Anormalidad, las Condiciones de Uso y el resultado del Análisis de Fallas.
-	**Redundancia**: el equipo puede o no tener sistemas que soporten su funcionamiento durante posibles fallas. Esta variable es independiente de las demás y afecta directamente la Confiabilidad.
-	**Confiabilidad**: puede ser alta, media o baja, y depende de las últimas dos variables descritas. Esta es la variable objeto del análisis. 

Para los nodos iniciales las probabilidades de cada uno de sus estados se presentan a continuación:
* **Historial de Anormalidad**: el estado **sin registro** tiene una probabilidad de 0.6, y **con registro** una probabilidad de 0.4.
* **Condiciones de Uso**: el estado de **uso normal** tiene una probabilidad de 0.8, y de **sobreuso** una probabilidad de 0.2.
* **Análisis de Fallas**: el estado **leve** tiene una probabilidad de 0.6, y **graves** una probabilidad de 0.4.
* **Redundancia**: el estado *con* tiene una probabilidad de 0.7, y *sin* una probabilidad de 0.3.

Probabilidades Condicionales
Para el nodo **No. de Fallas Latentes**, las probabilidades condicionales se presentan en la siguiente tabla.


Las probabilidades condicionales del nodo **Confiabilidad** dados sus nodos padres, se presentan a continuación.




De acuerdo con la información suministrada:

1.	Responda las siguientes preguntas:
a.	¿Tiene esta red estructuras en V? Si las tiene, ¿Qué implicaciones tienen en la influencia entre las variables de la red?

b.	¿Qué tipo de análisis considera es más relevante en este caso? ¿Causal, evidencial, intercausal? ¿Por qué?

c.	¿Qué independencias condicionales captura esta red?


2.	Construya gráficamente la Red Bayesiana que representa adecuadamente el problema.

3.	Halle la distribución de probabilidad de la variable nivel de Confiabilidad

In [ ]:
#objetos BayesianNetwork y TabularCPD
from pgmpy.models import BayesianNetwork
from pgmpy.factors.discrete import TabularCPD

In [ ]:
# Defina la estructura de la red
modelo = BayesianNetwork([("H", "FL"), ("CU", "FL"),("A", "FL"),("R", "C"),("FL", "C")])

In [ ]:
# Defina las CPDs:
cpd_H = TabularCPD(variable="H", variable_card=2, values=[[0.6], [0.4]])
cpd_CU = TabularCPD(variable="CU", variable_card=2, values=[[0.8], [0.2]])
cpd_A = TabularCPD(variable="A", variable_card=2, values=[[0.6], [0.4]])
cpd_R = TabularCPD(variable="R", variable_card=2, values=[[0.7], [0.3]])

cpd_FL = TabularCPD(
    variable="FL",
    variable_card=3,
    values=[
        [0.85, 0.8, 0.8, 0.7, 0.1, 0.05, 0.1, 0.02],
        [0.10, 0.15, 0.15, 0.2, 0.3, 0.1, 0.4, 0.03],
        [0.05, 0.05, 0.05, 0.1, 0.6, 0.85, 0.5, 0.95],
    ],
    evidence=["A", "CU", "H"],
    evidence_card=[2, 2, 2],
)

cpd_C = TabularCPD(
    variable="C",
    variable_card=3,
    values=[
        [0.95, 0.75, 0.4, 0.2, 0.1, 0],
        [0.05, 0.2, 0.3, 0.4, 0.3, 0.04],
        [0, 0.05, 0.3, 0.4, 0.6, 0.96],
    ],
    evidence=["R", "FL"],
    evidence_card=[2, 3],
)

print(cpd_FL)
print(cpd_C)

In [ ]:
# Asocie las CPDs con la estructura de red
modelo.add_cpds(cpd_H, cpd_CU, cpd_A, cpd_R, cpd_FL, cpd_C)

In [ ]:
#Distribución de riesgo de variable Confiabilidad
from pgmpy.inference import VariableElimination
inferencia = VariableElimination(modelo)
distribucion_C = inferencia.query(variables=['C'])
print(distribucion_C)

4. Si el equipo tiene registro del historial de anomalías, se encuentra en condiciones de sobreuso, la valoración de los daños presentados por el equipo previamente es leve y no tiene sistemas que soporten su funcionamiento durante posibles fallas, ¿cómo se ve afectada la distribución de riesgo del nivel de Confiabilidad?


In [ ]:
posterior_p = inferencia.query(["C"], evidence={"H": 1, "A": 1})
print(posterior_p)